## Trying conditional GANs in Faces image 32x32



In [1]:
#Import the libraries we will need.
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.contrib.slim as slim
import os
import scipy.misc
import scipy

To preprocess the face image

https://github.com/leblancfg/autocrop/blob/master/autocrop/autocrop.py

http://www.paulvangent.com/2016/04/01/emotion-recognition-with-python-opencv-and-a-face-dataset/

Install opencv

https://www.pyimagesearch.com/2015/07/20/install-opencv-3-0-and-python-3-4-on-ubuntu/

Auto-Encoder

https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/autoencoder.ipynb

awesome python library
https://pypi.python.org/pypi/tqdm

### Preparing training set

In [2]:
# Preparing training set
import glob
import random
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

class CK:
    
    def __init__(self):
        self.emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"] #Define emotions
        self.file_paths = []
        self.start_of_new_batch = 0
        self.height = 28
        self.width = 28
        self.channels = 1
        self.images_tensor=[]

        for emotion in self.emotions:
            self.file_paths.extend(glob.glob("./Face_data/CK+/dataset_28x28/%s/*" %emotion))

        random.shuffle(self.file_paths)
        size = len(self.file_paths)
        # Here generating a tensor of type string that include all the filename with png extention
        filename_queue  = tf.train.string_input_producer(self.file_paths)
        # Initializing a file Reader
        image_reader = tf.WholeFileReader()
        # Get all the files mentioned ie filename queue and
        # returns the  the file name and the pixelvalue in form of a tensor !
        _,imagefile= image_reader.read(filename_queue)
        image = tf.image.decode_png(imagefile, channels=self.channels)
        image.set_shape((self.height,self.width,self.channels))
        images = tf.train.shuffle_batch([image],
                                        batch_size=size,
                                        capacity=size,
                                        min_after_dequeue=0)

        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            # Coordinate the loading of image files.
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(coord=coord)

            # Get a batch of image tensors.
            self.images_tensor = sess.run(images)

            # Finish off the filename queue coordinator.
            coord.request_stop()
            coord.join(threads)
            
        # Convert from [0, 255] -> [0.0, 1.0].
        self.images_tensor = self.images_tensor.astype(np.float32)
        self.images_tensor = np.multiply(self.images_tensor, 1.0 / 255.0) 
        
#         img = mpimg.imread(paths_for_this_patch[0])
#         print(paths_for_this_patch[0])
#         im = images_tensor[0][0]
#         plt.figure(1)
#         plt.imshow(im.squeeze(), cmap='gray')
#         plt.figure(2)
#         plt.imshow(img, cmap='gray')
#         plt.show()
        
        
    def next_batch(self, size):
        
        if size > len(self.file_paths):
            size = len(self.file_paths)
        if size <= 0:
            return []
            
        # Get images for this batch
        this_patch = []
        start = self.start_of_new_batch
        end = (start + size - 1) % len(self.file_paths)
        if start > end:
            this_patch = self.images_tensor[start:len(self.file_paths)]
            this_patch = np.concatenate((this_patch,
                                         (self.images_tensor[0:end+1])))
        else:
            this_patch = self.images_tensor[start:end+1]
            
        self.start_of_new_batch = (start + size) % len(self.file_paths)
        

        return  this_patch

train_CK = CK()


### Helper Function

In [3]:
#This function performns a leaky relu activation, which is needed for the discriminator network.
def lrelu(x, leak=0.2, name="lrelu"):
     with tf.variable_scope(name):
         f1 = 0.5 * (1 + leak)
         f2 = 0.5 * (1 - leak)
         return f1 * x + f2 * abs(x)
    
#The below functions are taken from carpdem20's implementation https://github.com/carpedm20/DCGAN-tensorflow
#They allow for saving sample images from the generator to follow progress
def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)

def imsave(images, size, path):
    return scipy.misc.imsave(path, merge(images, size))

def inverse_transform(images):
    return (images+1.)/2.

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1]))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = image

    return img

def get_shape(tensor): # static shape
    return tensor.get_shape().as_list()

## Defining the Adversarial Networks

### Generator Network

In [4]:
def generator(z):
    
    # size: (out_height/(2*2*2)) * (out_width/(2*2*2)) * n
    zP = slim.fully_connected(z,4*4*512,normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_project',weights_initializer=initializer)
    zCon = tf.reshape(zP,[-1,4,4,512])
    
    gen1 = slim.convolution2d_transpose(\
        zCon,num_outputs=256,kernel_size=[4,4],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv1', weights_initializer=initializer)
    
    gen2 = slim.convolution2d_transpose(\
        gen1,num_outputs=128,kernel_size=[4,4],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv2', weights_initializer=initializer)
    
    gen3 = slim.convolution2d_transpose(\
        gen2,num_outputs=64,kernel_size=[4,4],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv3', weights_initializer=initializer)
    
    g_out = slim.convolution2d_transpose(\
        gen3,num_outputs=1,kernel_size=[32,32],padding="SAME",\
        biases_initializer=None,activation_fn=tf.nn.tanh,\
        scope='g_out', weights_initializer=initializer)
    
    return g_out

### Discriminator Network

The line of concat y with dis1 refer to:
https://github.com/Eyyub/tensorflow-cdcgan/blob/master/model/discriminator.py

Idea comes from IcGAN:
https://arxiv.org/abs/1611.06355

In [5]:
def discriminator(bottom, reuse=False):
    
    dis1 = slim.convolution2d(bottom,64,[4,4],stride=[2,2],padding="SAME",\
        biases_initializer=None,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv1',weights_initializer=initializer)    
    dis2 = slim.convolution2d(dis1,128,[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv2', weights_initializer=initializer)
    dis3 = slim.convolution2d(dis2,256,[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv3',weights_initializer=initializer)
    dis4 = slim.convolution2d(dis3,512,[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv4',weights_initializer=initializer)
    
    d_out = slim.fully_connected(slim.flatten(dis4),1,activation_fn=tf.nn.sigmoid,\
        reuse=reuse,scope='d_out', weights_initializer=initializer)
    
    return d_out

### Connecting them together

In [6]:
tf.reset_default_graph()

z_size = 100 #Size of z vector used for generator.

#This initializaer is used to initialize all the weights of the network.
initializer = tf.truncated_normal_initializer(stddev=0.02)

#These two placeholders are used for input into the generator and discriminator, respectively.
z_in = tf.placeholder(shape=[None,z_size],dtype=tf.float32) #Random vector
real_in = tf.placeholder(shape=[None,32,32,1],dtype=tf.float32) #Real images


Gz = generator(z_in) #Generates images from random z vectors
Dx = discriminator(real_in) #Produces probabilities for real images
Dg = discriminator(Gz, reuse=True) #Produces probabilities for generator images

#These functions together define the optimization objective of the GAN.
d_loss = -tf.reduce_mean(tf.log(Dx) + tf.log(1.-Dg)) #This optimizes the discriminator.
g_loss = -tf.reduce_mean(tf.log(Dg)) #This optimizes the generator.

tvars = tf.trainable_variables()

#The below code is responsible for applying gradient descent to update the GAN.
trainerD = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
trainerG = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
d_grads = trainerD.compute_gradients(d_loss,tvars[9:]) #Only update the weights for the discriminator network.
g_grads = trainerG.compute_gradients(g_loss,tvars[0:9]) #Only update the weights for the generator network.

update_D = trainerD.apply_gradients(d_grads)
update_G = trainerG.apply_gradients(g_grads)

### Training the network

In [7]:
batch_size = 128 #Size of image batch to apply at each iteration.
iterations = 50000 #Total number of iterations to use. 50000 -> 2hrs
sample_size = 25 # Number of sample to be view
sample_directory = './Face_DCGANs_32x32_figs' #Directory to save sample images from generator in.
model_directory = './Face_DCGANs_32x32_models' #Directory to save trained model to.

init = tf.initialize_all_variables()
saver = tf.train.Saver()
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:  
    sess.run(init)
    for i in range(iterations):
        zs = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32) #Generate a random z batch
        xs = train_CK.next_batch(batch_size) #Draw a sample batch from dataset.
        xs = (np.reshape(xs,[batch_size,28,28,1]) - 0.5) * 2.0 #Transform it to be between -1 and 1
        xs = np.lib.pad(xs, ((0,0),(2,2),(2,2),(0,0)),'constant', constant_values=(-1, -1)) #Pad the images so the are 64x64
        _,dLoss = sess.run([update_D,d_loss],feed_dict={z_in:zs,real_in:xs}) #Update the discriminator
        _,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs}) #Update the generator, twice for good measure.
        _,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs})
        if i % 10 == 0:
            print("Gen Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss))
            z2 = np.random.uniform(-1.0,1.0,size=[sample_size,z_size]).astype(np.float32) #Generate another z batch
            
            newZ = sess.run(Gz,feed_dict={z_in:z2}) #Use new z to get sample images from generator.
            if not os.path.exists(sample_directory):
                os.makedirs(sample_directory)
            #Save sample generator images for viewing training progress.
            save_images(np.reshape(newZ[0:25],[25,32,32]),[5,5],sample_directory+'/fig'+str(i)+'.png')
        if i % 1000 == 0 and i != 0:
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            saver.save(sess,model_directory+'/model-'+str(i)+'.cptk')
            print("Saved Model")

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Gen Loss: 0.191838 Disc Loss: 1.631
Gen Loss: 4.41648 Disc Loss: 0.339632
Gen Loss: 3.74178 Disc Loss: 0.06325
Gen Loss: 4.61306 Disc Loss: 0.0564478
Gen Loss: 4.80767 Disc Loss: 0.0227471
Gen Loss: 12.0233 Disc Loss: 0.044049
Gen Loss: 12.4069 Disc Loss: 0.677315
Gen Loss: 8.35899 Disc Loss: 0.264361
Gen Loss: 7.28586 Disc Loss: 0.0198269
Gen Loss: 5.61159 Disc Loss: 0.0318701
Gen Loss: 5.7894 Disc Loss: 0.00800841
Gen Loss: 6.22122 Disc Loss: 0.00652766
Gen Loss: 4.50938 Disc Loss: 0.0344985
Gen Loss: 4.98652 Disc Loss: 0.0126998
Gen Loss: 5.6063 Disc Loss: 0.0153261
Gen Loss: 5.67139 Disc Loss: 0.00775067
Gen Loss: 6.0592 Disc Loss: 0.00562331
Gen Loss: 6.8578 Disc Loss: 0.00452887
Gen Loss: 6.91792 Disc Loss: 0.00290506
Gen Loss: 6.2068 Disc Loss: 0.00377541
Gen Loss: 6.92151 Disc Loss: 0.00326164
Gen Loss: 7.10732 Disc Loss: 0.00257815
Gen Loss: 6.73179 Disc Loss: 0.0027706
Gen Loss: 7.15413 Disc Loss: 0.002

Gen Loss: 3.12194 Disc Loss: 0.238378
Gen Loss: 2.27298 Disc Loss: 0.223596
Gen Loss: 2.39897 Disc Loss: 0.27934
Gen Loss: 2.81107 Disc Loss: 0.114711
Gen Loss: 3.33306 Disc Loss: 0.108279
Gen Loss: 2.6801 Disc Loss: 0.113814
Gen Loss: 3.03046 Disc Loss: 0.121607
Gen Loss: 2.79848 Disc Loss: 0.0863724
Gen Loss: 4.08467 Disc Loss: 0.134721
Gen Loss: 3.62317 Disc Loss: 0.117431
Gen Loss: 2.96149 Disc Loss: 0.0720419
Gen Loss: 4.7631 Disc Loss: 0.0650192
Gen Loss: 3.6082 Disc Loss: 0.0840749
Gen Loss: 3.09477 Disc Loss: 0.0582255
Gen Loss: 1.57064 Disc Loss: 2.33565
Gen Loss: 2.32112 Disc Loss: 0.492846
Gen Loss: 3.09553 Disc Loss: 0.280432
Gen Loss: 2.99135 Disc Loss: 0.204718
Gen Loss: 4.3852 Disc Loss: 0.768077
Gen Loss: 3.47331 Disc Loss: 0.140438
Gen Loss: 3.05799 Disc Loss: 0.104517
Gen Loss: 6.44281 Disc Loss: 0.627423
Gen Loss: 2.98646 Disc Loss: 0.150927
Gen Loss: 2.35753 Disc Loss: 0.16979
Gen Loss: 2.91394 Disc Loss: 0.135624
Gen Loss: 3.50063 Disc Loss: 0.101156
Gen Loss: 3.25

Gen Loss: 4.02852 Disc Loss: 0.10669
Gen Loss: 4.61066 Disc Loss: 0.128241
Gen Loss: 5.64938 Disc Loss: 0.473652
Gen Loss: 3.02487 Disc Loss: 0.142647
Gen Loss: 4.66138 Disc Loss: 0.0681803
Gen Loss: 4.30595 Disc Loss: 0.0642134
Gen Loss: 3.82108 Disc Loss: 0.05568
Gen Loss: 6.35947 Disc Loss: 0.055814
Gen Loss: 3.94392 Disc Loss: 0.0487652
Gen Loss: 4.41361 Disc Loss: 0.041611
Gen Loss: 3.05303 Disc Loss: 0.0311772
Gen Loss: 4.90957 Disc Loss: 0.021661
Gen Loss: 4.0748 Disc Loss: 0.0378282
Gen Loss: 4.59603 Disc Loss: 0.0244219
Gen Loss: 5.94532 Disc Loss: 0.0391573
Gen Loss: 4.80169 Disc Loss: 0.0292996
Gen Loss: 8.60077 Disc Loss: 0.0582136
Gen Loss: 4.20951 Disc Loss: 0.0526702
Gen Loss: 3.7559 Disc Loss: 0.0640134
Gen Loss: 4.89721 Disc Loss: 0.0806164
Gen Loss: 6.58881 Disc Loss: 0.0794247
Gen Loss: 5.35943 Disc Loss: 0.108612
Gen Loss: 4.66144 Disc Loss: 0.0421565
Gen Loss: 5.3092 Disc Loss: 0.063951
Gen Loss: 5.56727 Disc Loss: 0.0410296
Gen Loss: 6.16511 Disc Loss: 0.0157009
G

Gen Loss: 7.61119 Disc Loss: 0.206485
Gen Loss: 8.19347 Disc Loss: 0.141325
Gen Loss: 2.76402 Disc Loss: 2.15161
Gen Loss: 6.28463 Disc Loss: 0.0534195
Gen Loss: 6.96313 Disc Loss: 0.0748972
Gen Loss: 4.89426 Disc Loss: 0.0548089
Gen Loss: 5.19445 Disc Loss: 0.0931815
Gen Loss: 4.43363 Disc Loss: 0.0396864
Gen Loss: 3.7286 Disc Loss: 0.0147594
Gen Loss: 3.75458 Disc Loss: 0.0493294
Gen Loss: 3.62378 Disc Loss: 0.0548321
Gen Loss: 3.08265 Disc Loss: 0.0406109
Gen Loss: 4.78704 Disc Loss: 0.059298
Gen Loss: 3.5971 Disc Loss: 0.0247263
Gen Loss: 4.08963 Disc Loss: 0.0378993
Gen Loss: 3.75312 Disc Loss: 0.0339096
Gen Loss: 4.22887 Disc Loss: 0.0417852
Gen Loss: 4.61075 Disc Loss: 0.0327285
Gen Loss: 5.1227 Disc Loss: 0.0471686
Gen Loss: 4.51496 Disc Loss: 0.0390246
Gen Loss: 4.37859 Disc Loss: 0.0317626
Gen Loss: 5.15708 Disc Loss: 0.0455146
Gen Loss: 5.17064 Disc Loss: 0.0170146
Gen Loss: 5.74855 Disc Loss: 0.0129835
Gen Loss: 5.07126 Disc Loss: 0.0313814
Gen Loss: 4.61225 Disc Loss: 0.02

Gen Loss: 5.58949 Disc Loss: 0.0267692
Gen Loss: 5.97372 Disc Loss: 0.0150169
Gen Loss: 7.38979 Disc Loss: 0.0035572
Gen Loss: 5.14001 Disc Loss: 0.00991432
Gen Loss: 5.82639 Disc Loss: 0.0226485
Gen Loss: 4.95879 Disc Loss: 0.00901208
Gen Loss: 5.77828 Disc Loss: 0.0244708
Gen Loss: 4.87681 Disc Loss: 0.0104187
Gen Loss: 4.68233 Disc Loss: 0.376878
Gen Loss: 5.46449 Disc Loss: 0.0230345
Gen Loss: 7.14579 Disc Loss: 0.0195488
Gen Loss: 4.97598 Disc Loss: 0.0298718
Gen Loss: 5.41498 Disc Loss: 0.0310656
Gen Loss: 5.86195 Disc Loss: 0.120932
Gen Loss: 5.36932 Disc Loss: 0.0153086
Gen Loss: 7.04597 Disc Loss: 0.00740439
Gen Loss: 5.11466 Disc Loss: 0.0390728
Gen Loss: 6.14832 Disc Loss: 0.00548884
Gen Loss: 6.12432 Disc Loss: 0.0210785
Gen Loss: 7.31233 Disc Loss: 0.0362235
Gen Loss: 6.43731 Disc Loss: 0.00777843
Gen Loss: 4.74345 Disc Loss: 0.0168187
Gen Loss: 5.37806 Disc Loss: 0.0190147
Gen Loss: 5.94239 Disc Loss: 0.00961794
Gen Loss: 4.35823 Disc Loss: 0.0201337
Gen Loss: 3.8394 Disc

...

Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan 

Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan 

Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Saved Model
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Ge

Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Saved Model
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Ge

Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan 

### Using a trained network